Here it goes

 First sequential is an MLP model. I used it previously and it performed rather good. 
 Now we will combine this with a convolutional neural network on the images

Convolutional Neural Networks are MLPs with a special structure. 

CNNs have repetitive blocks of neurons that are applied across space (for images) or time (for audio signals etc). For images, these blocks of neurons can be interpreted as 2D convolutional kernels, repeatedly applied over each patch of the image. For speech, they can be seen as the 1D convolutional kernels applied across time-windows. At training time, the weights for these repeated blocks are 'shared', i.e. the weight gradients learned over various image patches are averaged. 

The reason for choosing this special structure, is to exploit spatial or temporal invariance in recognition. For instance, a "dog" or a "car" may appear anywhere in the image. If we were to learn independent weights at each spatial or temporal location, it would take orders of magnitude more training data to train such an MLP. In over-simplified terms, for an MLP which didn't repeat weights across space, the group of neurons receiving inputs from the lower left corner of the image will have to learn to represent "dog" independently from the group of neurons connected to upper left corner, and correspondingly we will need enough images of dogs such that the network had seen several examples of dogs at each possible image location separately.

On top of this fundamental constraint, many special techniques and layers have been invented specially in the CNN context. So a latest deep CNN might look very different from a bare bones MLP, but the above is the difference in principle.




In [1]:
# Data Manipulation Libraries
import numpy as np
import pandas as pd

# Scikitlearn Libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit

# Keras Libraries
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import img_to_array, load_img

# Custom 
# Variable decleration. 

root = '..\input'
np.random.seed(2016)
split_random_state = 7
split = .9



Using TensorFlow backend.


In [22]:
# Below you are offered a switch on standardisation. 
# We start off with a function for loading training data.

def load_numeric_training(standardize=True):
    """
    Loads the pre-extracted features for the training data
    and returns a tuple of the image ids, the data, and the labels
    """
    # Read data from the CSV file
    # Pop actually does something more interesting, it also deletes the
    # from the dataframe. 
    
    train = pd.read_csv("train.csv")
    # This below populates the variable with one column and drops the rest
    ID = train.pop('id')
    

    # Since the labels are textual, so we encode them categorically
    y = train.pop('species')
    # Fit is to read in the data into the encoder
    # Tranform is to do the actual encoding
    y = LabelEncoder().fit(y).transform(y)
    
    # standardize the data by setting the mean to 0 and std to 1
    # standardScaler seems to follow the same formatting as labelEncoder
    X = StandardScaler().fit(train).transform(train) if standardize else train.values

    # Below is a tuple that has to be accepted in that order 
    
    return ID, X, y
    

In [17]:
# -------------------- Code not needed testing. 
ID, X, y = load_numeric_training(standardize=False)
X
# The array looks just like a spreadsheet will look. 

array([[ 0.007812,  0.023438,  0.023438, ...,  0.004883,  0.      ,
         0.025391],
       [ 0.005859,  0.      ,  0.03125 , ...,  0.000977,  0.039062,
         0.022461],
       [ 0.005859,  0.009766,  0.019531, ...,  0.      ,  0.020508,
         0.00293 ],
       ..., 
       [ 0.001953,  0.003906,  0.      , ...,  0.027344,  0.      ,
         0.001953],
       [ 0.      ,  0.      ,  0.046875, ...,  0.      ,  0.001953,
         0.00293 ],
       [ 0.023438,  0.019531,  0.03125 , ...,  0.023438,  0.025391,
         0.022461]])

In [1]:
# Next we have a function for loading testing data.

def load_numeric_test(standardize=True):
    """
    Loads the pre-extracted features for the test data
    and returns a tuple of the image ids, the data
    """
    test = pd.read_csv("input/test.csv")
    ID = test.pop('id')
    # standardize the data by setting the mean to 0 and std to 1
    test = StandardScaler().fit(test).transform(test) if standardize else test.values
    return ID, test


# Below looks like the resiation of one image. 

def resize_img(img, max_dim=96):
    """
    Resize the image to so the maximum side is of size max_dim
    Returns a new image of the right size
    """
    # Get the axis with the larger dimension
    # Recall that lambda is a throw-away, one-time, function 
    # Return the largest item in an iterable or the largest of two or more arguments.
    # 0,1 is the arguments you pass it, 0 is height, 1 is width. 
    max_ax = max((0, 1), key=lambda i: img.size[i])
    # Scale both axes so the image's largest dimension is max_dim
    scale = max_dim / float(img.size[max_ax])
    # This means that the proportional dimensions remain the same. 
    return img.resize((int(img.size[0] * scale), int(img.size[1] * scale)))


def load_image_data(ids, max_dim=96, center=True):
    """
    Takes as input an array of image ids and loads the images as numpy
    arrays with the images resized so the longest side is max-dim length.
    If center is True, then will place the image in the center of
    the output array, otherwise it will be placed at the top-left corner.
    """
    # Initialize the output array
    # NOTE: Theano users comment line below and
    # Return a new array of given shape and type, without initializing entries.
    # According to me this has 4 dimensions? 
    # You do get something like a multi-dimensional array. 
    X = np.empty((len(ids), max_dim, max_dim, 1))
    # X = np.empty((len(ids), 1, max_dim, max_dim)) # uncomment this
    # emumerate gives the real id and a number of the id.
    
    for i, idee in enumerate(ids):
        # Turn the image into an array
        x = resize_img(load_img(os.path.join('images/', str(idee) + '.jpg'), grayscale=True), max_dim=max_dim)
        # img to array is an existing method part of keras.
        # img has been resized but it has to put into array format. 
        x = img_to_array(x)
        # Get the corners of the bounding box for the image
        # NOTE: Theano users comment the two lines below and
        length = x.shape[0]
        width = x.shape[1]
        # length = x.shape[1] # uncomment this
        # width = x.shape[2] # uncomment this
        if center:
            # have to put into int for shape to understand. 
            # This is the code to position it to center
            h1 = int((max_dim - length) / 2)
            h2 = h1 + length
            w1 = int((max_dim - width) / 2)
            w2 = w1 + width
        else:
            # Now it will be left in the hoek
            h1, w1 = 0, 0
            h2, w2 = (length, width)
        # Insert into image matrix
        # NOTE: Theano users comment line below and
        X[i, h1:h2, w1:w2, 0:1] = x
        # X[i, 0:1, h1:h2, w1:w2] = x  # uncomment this
    # Scale the array values so they are between 0 and 1
    # It then rounds to a certain amount of decimals. 
    # Somehow they knew what the largest size was and they used this
    return np.around(X / 255.0)

# We only do the cross-validation on training data. 

def load_train_data(split=split, random_state=None):
    """
    Loads the pre-extracted feature and image training data and
    splits them into training and cross-validation.
    Returns one tuple for the training data and one for the validation
    data. Each tuple is in the order pre-extracted features, images,
    and labels.
    """
    # Load the pre-extracted features
    ID, X_num_tr, y = load_numeric_training()
    # Load the image data
    X_img_tr = load_image_data(ID)
    # Split them into validation and cross-validation
    sss = StratifiedShuffleSplit(n_splits=1, train_size=split, random_state=random_state)
    train_ind, test_ind = next(sss.split(X_num_tr, y))
    X_num_val, X_img_val, y_val = X_num_tr[test_ind], X_img_tr[test_ind], y[test_ind]
    X_num_tr, X_img_tr, y_tr = X_num_tr[train_ind], X_img_tr[train_ind], y[train_ind]
   
    return (X_num_tr, X_img_tr, y_tr), (X_num_val, X_img_val, y_val)


def load_test_data():
    """
    Loads the pre-extracted feature and image test data.
    Returns a tuple in the order ids, pre-extracted features,
    and images.
    """
    # Load the pre-extracted features
    ID, X_num_te = load_numeric_test()
    # Load the image data
    X_img_te = load_image_data(ID)
    return ID, X_num_te, X_img_te

print('Loading the training data...')
(X_num_tr, X_img_tr, y_tr), (X_num_val, X_img_val, y_val) = load_train_data(random_state=split_random_state)
y_tr_cat = to_categorical(y_tr)
y_val_cat = to_categorical(y_val)
print('Training data loaded!')

Using TensorFlow backend.


CalledProcessError: Command '['ls', '../input']' returned non-zero exit status 1

In [16]:
#------------------------------------------------------- Data augmentation
# This taks inludes random image rotation and zoom

from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator, array_to_img

# A little hacky piece of code to get access to the indices of the images...
# ...the data augmenter is working with.
class ImageDataGenerator2(ImageDataGenerator):
    def flow(self, X, y=None, batch_size=32, shuffle=True, seed=None,
             save_to_dir=None, save_prefix='', save_format='jpeg'):
        return NumpyArrayIterator2(
            X, y, self,
            batch_size=batch_size, shuffle=shuffle, seed=seed,
            dim_ordering=self.dim_ordering,
            save_to_dir=save_to_dir, save_prefix=save_prefix, save_format=save_format)


class NumpyArrayIterator2(NumpyArrayIterator):
    def next(self):
        # for python 2.x.
        # Keeps under lock only the mechanism which advances
        # the indexing of each batch
        # see http://anandology.com/blog/using-iterators-and-generators/
        with self.lock:
            # We changed index_array to self.index_array
            self.index_array, current_index, current_batch_size = next(self.index_generator)
        # The transformation of images is not under thread lock so it can be done in parallel
        batch_x = np.zeros(tuple([current_batch_size] + list(self.X.shape)[1:]))
        for i, j in enumerate(self.index_array):
            x = self.X[j]
            x = self.image_data_generator.random_transform(x.astype('float32'))
            x = self.image_data_generator.standardize(x)
            batch_x[i] = x
        if self.save_to_dir:
            for i in range(current_batch_size):
                img = array_to_img(batch_x[i], self.dim_ordering, scale=True)
                fname = '{prefix}_{index}_{hash}.{format}'.format(prefix=self.save_prefix,
                                                                  index=current_index + i,
                                                                  hash=np.random.randint(1e4),
                                                                  format=self.save_format)
                img.save(os.path.join(self.save_to_dir, fname))
        if self.y is None:
            return batch_x
        batch_y = self.y[self.index_array]
        return batch_x, batch_y

print('Creating Data Augmenter...')
imgen = ImageDataGenerator2(
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')
imgen_train = imgen.flow(X_img_tr, y_tr_cat, seed=np.random.randint(1, 10000))
print('Finished making data augmenter...')





Creating Data Augmenter...
Finished making data augmenter...


For basic neural network architectures we can use Keras's Sequential API, but since we need to build a model that takes two different inputs (image and pre-extracted features) in two different locations in the model, we won't be able to use the Sequential API. Instead, we'll be using the Functional API. This API is just as straightforward, but instead of having a model we add layers to, we'll instead be passing an array through a layer, and passing that output through another layer, and so on. You can think of each layer as a function and the array we give it as its argument. Click here for more info about the functional API.

In [17]:
#---------------------------Combining the Image CNN with the Pre-Extracted Features MLP¶

from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Convolution2D, MaxPooling2D, Flatten, Input, merge


def combined_model():

    # Define the image input
    image = Input(shape=(96, 96, 1), name='image')
    # Pass it through the first convolutional layer
    x = Convolution2D(8, 5, 5, input_shape=(96, 96, 1), border_mode='same')(image)
    x = (Activation('relu'))(x)
    x = (MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))(x)

    # Now through the second convolutional layer
    x = (Convolution2D(32, 5, 5, border_mode='same'))(x)
    x = (Activation('relu'))(x)
    x = (MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))(x)

    # Flatten our array
    x = Flatten()(x)
    # Define the pre-extracted feature input
    numerical = Input(shape=(192,), name='numerical')
    # Concatenate the output of our convnet with our pre-extracted feature input
    concatenated = merge([x, numerical], mode='concat')

    # Add a fully connected layer just like in a normal MLP
    x = Dense(100, activation='relu')(concatenated)
    x = Dropout(.5)(x)

    # Get the final output
    out = Dense(99, activation='softmax')(x)
    # How we create models with the Functional API
    model = Model(input=[image, numerical], output=out)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    return model

print('Creating the model...')
model = combined_model()
print('Model created!')






Creating the model...
Model created!


Now we're finally ready to actually train the model! Running on Kaggle will take a while. It's MUCH faster to run it locally if you have a GPU, or on an AWS instance with a GPU.

In [21]:
from keras.callbacks import ModelCheckpoint
from keras.models import load_model


def combined_generator(imgen, X):
    """
    A generator to train our keras neural network. It
    takes the image augmenter generator and the array
    of the pre-extracted features.
    It yields a minibatch and will run indefinitely
    """
    while True:
        for i in range(X.shape[0]):
            # Get the image batch and labels
            batch_img, batch_y = next(imgen)
            # This is where that change to the source code we
            # made will come in handy. We can now access the indicies
            # of the images that imgen gave us.
            x = X[imgen.index_array]
            yield [batch_img, x], batch_y

# autosave best Model
best_model_file = "leafnet.h5"
best_model = ModelCheckpoint(best_model_file, monitor='val_loss', verbose=1, save_best_only=True)

print('Training model...')
history = model.fit_generator(combined_generator(imgen_train, X_num_tr),
                              samples_per_epoch=X_num_tr.shape[0],
                              nb_epoch=1000,
                              validation_data=([X_img_val, X_num_val], y_val_cat),
                              nb_val_samples=X_num_val.shape[0],
                              verbose=0,
                              callbacks=[best_model])

print('Loading the best model...')
model = load_model(best_model_file)
print('Best Model loaded!')

Training model...
Epoch 00000: val_loss improved from inf to 0.00039, saving model to leafnet.h5
Epoch 00001: val_loss improved from 0.00039 to 0.00037, saving model to leafnet.h5
Epoch 00002: val_loss did not improve
Epoch 00003: val_loss did not improve
Epoch 00004: val_loss did not improve
Epoch 00005: val_loss did not improve
Epoch 00006: val_loss did not improve
Epoch 00007: val_loss improved from 0.00037 to 0.00025, saving model to leafnet.h5
Epoch 00008: val_loss did not improve
Epoch 00009: val_loss did not improve
Epoch 00010: val_loss did not improve
Epoch 00011: val_loss did not improve
Epoch 00012: val_loss did not improve
Epoch 00013: val_loss did not improve
Epoch 00014: val_loss did not improve
Epoch 00015: val_loss did not improve
Epoch 00016: val_loss did not improve
Epoch 00017: val_loss did not improve
Epoch 00018: val_loss did not improve
Epoch 00019: val_loss did not improve
Epoch 00020: val_loss did not improve
Epoch 00021: val_loss did not improve
Epoch 00022: va

KeyboardInterrupt: 

In [ ]:
## we need to consider the loss for final submission to leaderboard
## print(history.history.keys())
print('val_acc: ',max(history.history['val_acc']))
print('val_loss: ',min(history.history['val_loss']))
print('train_acc: ',max(history.history['acc']))
print('train_loss: ',min(history.history['loss']))

print()
print("train/val loss ratio: ", min(history.history['loss'])/min(history.history['val_loss']))

In [ ]:
# summarize history for loss
## Plotting the loss with the number of iterations

plt.semilogy(history.history['loss'])
plt.semilogy(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
## Plotting the error with the number of iterations
## With each iteration the error reduces smoothly
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [22]:
# Get the names of the column headers
LABELS = sorted(pd.read_csv("input/train.csv").species.unique())

index, test, X_img_te = load_test_data()

yPred_proba = model.predict([X_img_te, test])

# Converting the test predictions in a dataframe as depicted by sample submission
yPred = pd.DataFrame(yPred_proba,index=index,columns=LABELS)

print('Creating and writing submission...')
fp = open('submit.csv', 'w')
fp.write(yPred.to_csv())
print('Finished writing submission')
# Display the submission
yPred.tail()

Creating and writing submission...
Finished writing submission


,Acer_Capillipes,Acer_Circinatum,Acer_Mono,Acer_Opalus,Acer_Palmatum,Acer_Pictum,Acer_Platanoids,Acer_Rubrum,Acer_Rufinerve,Acer_Saccharinum,...,Salix_Fragilis,Salix_Intergra,Sorbus_Aria,Tilia_Oliveri,Tilia_Platyphyllos,Tilia_Tomentosa,Ulmus_Bergmanniana,Viburnum_Tinus,Viburnum_x_Rhytidophylloides,Zelkova_Serrata
id,,,,,,,,,,,,,,,,,,,,,
1576,9.732460e-23,1.000000e+00,7.580490e-22,2.918674e-21,1.475046e-11,4.965136e-21,1.489809e-29,2.351745e-19,5.251790e-16,8.054925e-17,...,1.448120e-29,2.640295e-29,1.381857e-31,6.660142e-23,4.477618e-32,1.034051e-25,2.274980e-24,7.207536e-32,2.902506e-30,1.728592e-13
1577,3.179354e-21,2.761599e-19,1.428463e-28,4.436568e-12,3.785086e-25,9.450088e-32,8.500857e-23,1.481659e-14,1.648460e-12,1.940630e-24,...,8.929313e-21,6.713418e-26,1.783730e-11,3.478057e-18,5.390251e-10,4.342291e-09,1.477519e-17,1.087486e-18,8.494681e-27,1.845044e-15
1579,1.929810e-14,8.584706e-23,3.179017e-27,1.185202e-24,2.478193e-19,3.264831e-16,1.233863e-34,4.201948e-20,3.336759e-26,1.137493e-14,...,6.611348e-35,2.137328e-29,9.296880e-23,4.154239e-23,4.602525e-20,8.577914e-34,4.263994e-31,6.403094e-22,2.787292e-22,1.259395e-18
1580,1.540434e-22,5.426165e-19,7.103119e-21,1.770784e-15,1.507297e-19,2.437834e-29,8.464697e-19,8.985027e-13,2.677789e-18,1.005978e-26,...,3.181805e-21,2.801440e-20,3.111767e-34,2.949946e-12,2.936691e-27,1.104438e-19,4.946997e-24,1.722264e-16,4.367362e-30,2.977807e-21
1583,0.000000e+00,4.794486e-23,3.849859e-25,5.476025e-28,8.038132e-25,6.674886e-18,4.537171e-20,6.503471e-25,1.648464e-29,7.302382e-31,...,6.600870e-26,6.455269e-34,9.636244e-34,1.585682e-28,4.156420e-26,1.464748e-30,5.361583e-32,9.176537e-34,3.752899e-38,9.067588e-26
